## **STEP 1: Importing packages and Reading Data**    
  

### **IMPORTING PACKAGES**

**Below is the steps we would go through to load, view and visualize any xml data into a pandas dataframe.**  


In [1]:
import xml.etree.cElementTree as et   # required to extract data from xml format

**We add python packages we require.**  

In [2]:
import numpy  as np                   # easy to play with arrays etc.
import pandas as pd                   # required to load and read data and put in dataframe.
import matplotlib.pyplot as plt       # required for data visualization purposes.
import seaborn as sns                 # required for data visualization purposes.
import plotly.plotly as py            # required for data visualization purposes.
import plotly.graph_objs as go        # required for data visualization purposes.
import re
from IPython.display import display, HTML
from plotly.offline import init_notebook_mode, iplot
init_notebook_mode(connected=True)

### **READING DATA**  

**To read data in the form of .xml, you need et.parse()**  
**We also need to get the root for this we use .getroot()**

**XML files are all strcutured differently, so you need to explore a bit**  

**https://catalog.data.gov/dataset/age-adjusted-death-rates-for-the-top-10-leading-causes-of-death-united-states-2013 is source of data**  

In [3]:
parser=RDF.Parser(name="ntriples") #as name for parser you can use ntriples, turtle, rdfxml, ...
model=RDF.Model()
stream=parser.parse_into_model(model,"file://file_path")
for triple in model:
    print("triple.subject, triple.predicate, triple.objectparsedXML = et.parse('rows.xml')")
root = parsedXML.getroot()

dictionary_to_df = {}

for neighbor in root.findall("row/row"):
    #print neighbor.tag,neighbor.attrib
    for child in neighbor:
        #The below two lines should be added to make sure all the arrays in the dictionary stay the same length.
        #If you remove those lines and run, make sure to run the last two lines in this cell to see the sizes of arrays.
        if child.tag == 'predicted_value' or child.tag == 'footnote':
            continue
            
        if(child.tag not in dictionary_to_df):
            dictionary_to_df[child.tag]=[child.text] 
        else:            
            dictionary_to_df[child.tag].append(child.text)
        
# for key, array in  dictionary_to_df.items():
    # print key, len(array)

NameError: name 'RDF' is not defined

**STEP 3: PUTTING THE DATA IN A PANDAS DATAFRAME**  
  

**We use pd.DataFrame() to put the data we extracted into a pandas df format**

In [ ]:
data = pd.DataFrame.from_dict(dictionary_to_df)
display(HTML(data.head().to_html()))

**STEP 4: DECIDING WHICH COLUMNS MATTER**  
  

**There are several columns with data that is redundant. Essentially we pick the columns we want, in the order that we want and discard the rest**

In [ ]:
data = data[['state', 'year', 'month', 'percent_complete', 'percent_pending_investigation','data_value']]
print ("\t\t\t The Final Dataset")
display(HTML(data.head().to_html()))

**STEP 5: VIEWING THE FIRST FEW ROWS**  
  

**To see the first few rows of the data and make sure we read it in correctly, we use .head()**


In [ ]:
display(HTML(data.head(15).to_html()))

**STEP 6: GET BASIC INFORMATION**  
  
**To get basic info from the dataset, we use .info()**

In [ ]:
data.info()

**STEP 7: SEE FURTHER DETAILS**  
  
**To get datatypes of each column, we can use .dtypes**  

**To get more details about each column, we can use .describe()**  
  
The reason we only get data from 3 columns is because the rest have commas in them which need to be removed
We can deal with this later while cleaning.

In [ ]:
print (data.dtypes)
print ()"\n")
display(HTML(data.describe().to_html()))

**STEP 8: COUNT NUMBER OF EMPTY VALUES IN COLUMN**  
   
**We can check the number of null values a column has by using .isnull().sum()**  
  
**For example, here, Climate has the most null values **  

In [ ]:
print (data.isnull().sum())

**STEP 9: SEE NUMBER OF UNIQUE VALUES IN COLUMN**  
  
**It is useful to see the number of unique values in each column using .nunique()**  
  
**Here we see state year and month have a good number of unique values to order by, therefore we can group by these columns and make good visualizations**  

In [ ]:
print (data.nunique())

**STEP 10: NUMBER OF OCCURANCES OF EACH VALUE IN COLUMN**  
  
**A good way to visualize data of a column you wish to group by is to use .value_counts()**  
  
**It gives a clear picture of how many would be in each group etc.** 

In [ ]:
year = data.year.value_counts()
print (year)
print ("\n")

month = data.month.value_counts()
print (month)
print ()"\n")

state = data.state.value_counts()
print (state.head(15))

**STEP 10: CLEANING DATA**  
  
**To see more from the data it has to be cleaned. Cleaning data is usally unique to each dataset.**  

**In this instance, we change column dtypes **

In [ ]:
for i in data.columns:
    if i== 'state' or i=='year' or i=='month':
        
        data[i] = data[i].str.strip().astype('category')
    else:      
        data[i] = data[i].str.strip('+-').astype(float)

**Now after cleaning, lets look at the new data types and the mean, std, min, max etc. of all the columns again**

In [ ]:
print (data.dtypes)
print ()"\n")
data.describe()

**STEP 11: PLOT WHOLE DATESET**  
  
**Let us try to visualize all the data at once**  

In [ ]:
a = data.plot()
plt.show()

**STEP 12: REARRANGING DATA**  
  
**Given that the data we are using is structured differently. It would make sense to group data by the state and month.**
**Therefore we create 2 new dataframes.**  
The second i simply an aveage of all the columns in the first one.  
**This makes it easier to visualize and analyze the data.**

In [ ]:
grouped = data.groupby(["state","month"], sort=False)
new_data= grouped['data_value'].agg(np.mean)   
new_data = new_data.unstack(level=-1)
display(HTML(new_data.head().to_html()))

col = new_data.loc[: , "January":"December"]
new_stateavg_data=pd.DataFrame()
new_stateavg_data['All_Month_Average'] =  col.mean(axis=1)
new_stateavg_data = new_stateavg_data.drop(['US'])
display(HTML(new_stateavg_data.head().to_html()))
display(HTML(new_stateavg_data.describe().to_html()))

**STEP 13: HEATMAP OF CORRELATION BETWEEN COLUMNS**  
  
**When two sets of data are strongly linked together we say they have a High Correlation. To see corr between all the columns, we use .corr()**

In [ ]:
f,ax = plt.subplots(figsize=(15, 13))
sns.heatmap(new_data.corr(), annot=True, ax=ax)
plt.show()

**STEP 14: COMPLEX VISUALIZATION**  
  
**Next we want to be able to make more complex visualizations to better understand code** 
  
**To start with, lets use plotly.graph_objs to visualize data on a USA map.**  
   
This can be repeated for the other dataframes.

In [ ]:
data2 = dict(
        type='choropleth',
        locations = list(new_stateavg_data.index),
        z = new_stateavg_data.All_Month_Average,
        locationmode = 'USA-states',
        text = list(new_stateavg_data.index),colorbar = {'title':'Total Deaths', 'tickmode' : 'array',},
    )

layout = dict(title='Location Projections',
    geo = dict(projection={'type':'albers usa'}))
    
fig = dict( data=[data2], layout=layout )
iplot( fig )